In [4]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt


start = time.time()
#print(start)
print(datetime.datetime.now())

#pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'

file_path = '/home/vimalkumar/Documents/stewart/Pdf files'
excel_path = '/home/vimalkumar/Documents/stewart/excel_output'

for pdf_file in sorted(os.listdir(file_path)):
    print(pdf_file)
    in_path = os.path.join(file_path,pdf_file)
    out_path = os.path.join('/home/vimalkumar/Documents/stewart/Images',
                            str(os.path.splitext(pdf_file)[0]))

    os.makedirs(out_path)

    doc = fitz.open(in_path)
    
    print('No of Images: ',len(doc))

    for i in range(len(doc)):
        doc = fitz.open(in_path)
        page = doc.load_page(i)  # number of page
        pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
        file_name = str(i+1000 + 1)       
        pix.pil_save(os.path.join(out_path,file_name+'.jpeg'))

    dic = {0:'Blank',1:'Not Blank'}

    inception_model = keras.models.load_model('stewart_inception_model.h5')
    vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')
    vgg19_model = keras.models.load_model('stewart_vgg19_model_1.h5')


    pdf_name = []
    file_name = []
    inception_result = []
    vgg_result = []
    ocr_result = []
    vgg19_result = []


    cnt = 1

    for im in sorted(os.listdir(out_path)):
        pdf_name.append(pdf_file)
        #print(cnt)
        print(cnt,'  ',im)
        cnt = cnt + 1
        img = image.load_img(os.path.join(out_path,im),
                         target_size=(224,224))
        #plt.imshow(img)
        #plt.show()

        x = image.img_to_array(img)
        x = x/255
        x = np.expand_dims(x,axis=0)
        images = np.vstack([x])
        val = inception_model.predict(images)
        #print(val)

        a=np.argmax(val,axis=1)
        output_inception = dic[a[0]]
        #print(output_inception)
        file_name.append(im)

        #image_read = cv2.imread(os.path.join(in_path,im),0)
        pil_read = Image.open(os.path.join(out_path,im))
        pil_read_gray = ImageOps.grayscale(pil_read)
        img_rgb = np.array(pil_read_gray)


        _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

        nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
        sizes = stats[1:, -1] #get CC_STAT_AREA component
        img2 = np.zeros((labels.shape), np.uint8)

        for i in range(0, nlabels - 1):
            if sizes[i] >= 50:   #filter small dotted regions
                img2[labels == i + 1] = 255

        res = cv2.bitwise_not(img2)
    #     data = Image.fromarray(res)
    #     enh_con = ImageEnhance.Contrast(data)
    #     contrast = 3.0
    #     image_contrasted = enh_con.enhance(contrast)
        image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
        image_data_lower = " ".join(image_data.splitlines()).lower()

        stats = ImageStat.Stat(pil_read)
        #print(stats)

        y = []
        for band,name in enumerate(pil_read.getbands()):
            x=sum(stats.extrema[band])
            y.append(x)
        if sum(y)==765:
            #print('good')
            image_data_1 = pytesseract.image_to_string(pil_read_gray)
            image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
            image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

        else:
            #print('dark')
            image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
            image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
            image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
            #print(image_data_1_lower)



        if output_inception == 'Blank':
            inception_result.append(output_inception)
        else:


            fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
            fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

            if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                inception_result.append('New Document')
            else:
                inception_result.append('Not Blank')


        val_1 = vgg_model.predict(images)
        #print(val_1)

        a_1=np.argmax(val_1,axis=1)
        output_vgg = dic[a_1[0]]
        #print(output_vgg)
        #file_name.append(im)
        #vgg_result.append(output_vgg)

        if output_vgg == 'Blank':
            vgg_result.append(output_vgg)
        else:
            #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
            #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
            #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

            fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
            fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

            if fuzz_ratio_vgg > 80 or fuzz_ratio_vgg_1 > 80:
                vgg_result.append('New Document')
            else:
                vgg_result.append('Not Blank')
                
                
        val_2 = vgg19_model.predict(images)
        #print(val_1)

        a_2=np.argmax(val_2,axis=1)
        output_vgg19 = dic[a_2[0]]
        #print(output_vgg)
        #file_name.append(im)
        #vgg_result.append(output_vgg)

        if output_vgg19 == 'Blank':
            vgg19_result.append(output_vgg19)
        else:
            #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
            #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
            #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

            fuzz_ratio_vgg19 = fuzz.partial_ratio(image_data_lower,'new document to follow')
            fuzz_ratio_vgg19_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

            if fuzz_ratio_vgg19 > 80 or fuzz_ratio_vgg19_1 > 80:
                vgg19_result.append('New Document')
            else:
                vgg19_result.append('Not Blank')



        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

        if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
            ocr_result.append('New Document')

        elif len(image_data_1_lower) < 15:
            ocr_result.append('Blank')

        else:
            ocr_result.append('Not Blank')


    df = pd.DataFrame({'PDF':pdf_name,'Images':file_name,'Inception_result':inception_result,
                       'VGG_result':vgg_result,'VGG19_result':vgg19_result,
                       'OCR_Result':ocr_result})
    #display(df)
    df['Confidence']=(df['Inception_result']==df['VGG_result'])&(df['Inception_result']==df['VGG19_result'])&(df['Inception_result']==df['OCR_Result'])

    df['status'] = df['Inception_result']
    df['Confidence'] = df['Confidence'].replace(True,'Yes')
    df['Confidence'] = df['Confidence'].replace(False,'No')

    pd.set_option('display.max_rows',1000)
    display(df)
    
    final_df = df[['PDF','Images','status','Confidence']]
    display(final_df)

    final_df.to_excel(os.path.join(excel_path,str(os.path.splitext(pdf_file)[0])+'.xlsx'))            

end = time.time()

print(end - start) 
    



2022-02-14 13:18:58.225890
Batch 1.pdf
No of Images:  235
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg
25    1025.jpeg
26    1026.jpeg
27    1027.jpeg
28    1028.jpeg
29    1029.jpeg
30    1030.jpeg
31    1031.jpeg
32    1032.jpeg
33    1033.jpeg
34    1034.jpeg
35    1035.jpeg
36    1036.jpeg
37    1037.jpeg
38    1038.jpeg
39    1039.jpeg
40    1040.jpeg
41    1041.jpeg
42    1042.jpeg
43    1043.jpeg
44    1044.jpeg
45    1045.jpeg
46    1046.jpeg
47    1047.jpeg
48    1048.jpeg
49    1049.jpeg
50    1050.jpeg
51    1051.jpeg
52    1052.jpeg
53    1053.jpeg
54    1054.jpeg
55    1055.jpeg
56    1056.jpeg
57    1057.jpeg
58    1058.jpeg
59    1059.jpeg
60    1

,PDF,Images,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,Batch 1.pdf,1001.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
1,Batch 1.pdf,1002.jpeg,New Document,New Document,New Document,New Document,Yes,New Document
2,Batch 1.pdf,1003.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,Batch 1.pdf,1004.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 1.pdf,1005.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 1.pdf,1006.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
6,Batch 1.pdf,1007.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
7,Batch 1.pdf,1008.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
8,Batch 1.pdf,1009.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 1.pdf,1010.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,PDF,Images,status,Confidence
0,Batch 1.pdf,1001.jpeg,Not Blank,Yes
1,Batch 1.pdf,1002.jpeg,New Document,Yes
2,Batch 1.pdf,1003.jpeg,Not Blank,Yes
3,Batch 1.pdf,1004.jpeg,Not Blank,Yes
4,Batch 1.pdf,1005.jpeg,Not Blank,Yes
5,Batch 1.pdf,1006.jpeg,Not Blank,Yes
6,Batch 1.pdf,1007.jpeg,Not Blank,Yes
7,Batch 1.pdf,1008.jpeg,Not Blank,Yes
8,Batch 1.pdf,1009.jpeg,Not Blank,Yes
9,Batch 1.pdf,1010.jpeg,Not Blank,Yes


Batch 10.pdf
No of Images:  284
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg
25    1025.jpeg
26    1026.jpeg
27    1027.jpeg
28    1028.jpeg
29    1029.jpeg
30    1030.jpeg
31    1031.jpeg
32    1032.jpeg
33    1033.jpeg
34    1034.jpeg
35    1035.jpeg
36    1036.jpeg
37    1037.jpeg
38    1038.jpeg
39    1039.jpeg
40    1040.jpeg
41    1041.jpeg
42    1042.jpeg
43    1043.jpeg
44    1044.jpeg
45    1045.jpeg
46    1046.jpeg
47    1047.jpeg
48    1048.jpeg
49    1049.jpeg
50    1050.jpeg
51    1051.jpeg
52    1052.jpeg
53    1053.jpeg
54    1054.jpeg
55    1055.jpeg
56    1056.jpeg
57    1057.jpeg
58    1058.jpeg
59    1059.jpeg
60    1060.jpeg
61    1061.jpeg
6

,PDF,Images,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,Batch 10.pdf,1001.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
1,Batch 10.pdf,1002.jpeg,New Document,New Document,New Document,New Document,Yes,New Document
2,Batch 10.pdf,1003.jpeg,Blank,Blank,Blank,Blank,Yes,Blank
3,Batch 10.pdf,1004.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 10.pdf,1005.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 10.pdf,1006.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
6,Batch 10.pdf,1007.jpeg,Blank,Blank,Blank,Blank,Yes,Blank
7,Batch 10.pdf,1008.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
8,Batch 10.pdf,1009.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 10.pdf,1010.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


...

,PDF,Images,status,Confidence
0,Batch 7.pdf,1001.jpeg,Not Blank,Yes
1,Batch 7.pdf,1002.jpeg,New Document,Yes
2,Batch 7.pdf,1003.jpeg,Not Blank,Yes
3,Batch 7.pdf,1004.jpeg,Not Blank,Yes
4,Batch 7.pdf,1005.jpeg,Not Blank,Yes
5,Batch 7.pdf,1006.jpeg,Not Blank,No
6,Batch 7.pdf,1007.jpeg,Not Blank,Yes
7,Batch 7.pdf,1008.jpeg,Not Blank,Yes
8,Batch 7.pdf,1009.jpeg,Not Blank,Yes
9,Batch 7.pdf,1010.jpeg,Not Blank,Yes


Batch 9.pdf
No of Images:  261
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg
25    1025.jpeg
26    1026.jpeg
27    1027.jpeg
28    1028.jpeg
29    1029.jpeg
30    1030.jpeg
31    1031.jpeg
32    1032.jpeg
33    1033.jpeg
34    1034.jpeg
35    1035.jpeg
36    1036.jpeg
37    1037.jpeg
38    1038.jpeg
39    1039.jpeg
40    1040.jpeg
41    1041.jpeg
42    1042.jpeg
43    1043.jpeg
44    1044.jpeg
45    1045.jpeg
46    1046.jpeg
47    1047.jpeg
48    1048.jpeg
49    1049.jpeg
50    1050.jpeg
51    1051.jpeg
52    1052.jpeg
53    1053.jpeg
54    1054.jpeg
55    1055.jpeg
56    1056.jpeg
57    1057.jpeg
58    1058.jpeg
59    1059.jpeg
60    1060.jpeg
61    1061.jpeg
62

,PDF,Images,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,Batch 9.pdf,1001.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
1,Batch 9.pdf,1002.jpeg,New Document,New Document,New Document,New Document,Yes,New Document
2,Batch 9.pdf,1003.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,Batch 9.pdf,1004.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 9.pdf,1005.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 9.pdf,1006.jpeg,Blank,Blank,Blank,Blank,Yes,Blank
6,Batch 9.pdf,1007.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
7,Batch 9.pdf,1008.jpeg,Blank,Blank,Blank,Blank,Yes,Blank
8,Batch 9.pdf,1009.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 9.pdf,1010.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,PDF,Images,status,Confidence
0,Batch 9.pdf,1001.jpeg,Not Blank,Yes
1,Batch 9.pdf,1002.jpeg,New Document,Yes
2,Batch 9.pdf,1003.jpeg,Not Blank,Yes
3,Batch 9.pdf,1004.jpeg,Not Blank,Yes
4,Batch 9.pdf,1005.jpeg,Not Blank,Yes
5,Batch 9.pdf,1006.jpeg,Blank,Yes
6,Batch 9.pdf,1007.jpeg,Not Blank,Yes
7,Batch 9.pdf,1008.jpeg,Blank,Yes
8,Batch 9.pdf,1009.jpeg,Not Blank,Yes
9,Batch 9.pdf,1010.jpeg,Not Blank,Yes


8679.394520759583


# 6 pdf( 2394 pages) = 6471 seconds(2.70 seconds / page)
# 16 pdf(3676 pages) = 8679 seconds(2.36 seconds/page)

# Zip with selecting pdf files



In [2]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
import zipfile
from pathlib import Path

from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt


start = time.time()
#print(start)
print(datetime.datetime.now())

#pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'
zip_file = '1507-1513.zip'

zip_path = '/home/vimalkumar/Documents/stewart/Pdf files'

os.makedirs(os.path.join(zip_path,os.path.splitext(zip_file)[0]))

with zipfile.ZipFile(os.path.join("/home/vimalkumar/Documents/stewart/zip",zip_file),"r") as zip_ref:
    #print(zip_ref)
    zip_ref.extractall(os.path.join(zip_path,os.path.splitext(zip_file)[0]))
    
    
file_path = '/home/vimalkumar/Documents/stewart/Pdf files'
images_path = '/home/vimalkumar/Documents/stewart/Images'
excel_path = '/home/vimalkumar/Documents/stewart/excel_output'

os.makedirs(os.path.join(images_path,os.path.splitext(zip_file)[0]))
os.makedirs(os.path.join(excel_path,os.path.splitext(zip_file)[0]))



for pdf_file in sorted(os.listdir(os.path.join(file_path,os.path.splitext(zip_file)[0]))):
    if pdf_file.endswith('.pdf')or pdf_file.endswith('.PDF')or pdf_file.endswith('.Pdf'):
        print(pdf_file)
        in_path = os.path.join(file_path,os.path.splitext(zip_file)[0],pdf_file)
        out_path = os.path.join(images_path,os.path.splitext(zip_file)[0],
                                str(os.path.splitext(pdf_file)[0]))

        os.makedirs(out_path)
        
        file_size=Path(os.path.join(file_path,os.path.splitext(zip_file)[0],pdf_file)).stat().st_size
        
        if file_size != 0:
            doc = fitz.open(in_path)

            print('No of Images: ',len(doc))



            for i in range(len(doc)):
                doc = fitz.open(in_path)
                page = doc.load_page(i)  # number of page
                pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
                file_name = str(i+1000 + 1)       
                pix.pil_save(os.path.join(out_path,file_name+'.jpeg'))

            dic = {0:'Blank',1:'Not Blank'}

            inception_model = keras.models.load_model('stewart_inception_model.h5')
            vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')
            vgg19_model = keras.models.load_model('stewart_vgg19_model_1.h5')


            pdf_name = []
            file_name = []
            inception_result = []
            vgg_result = []
            ocr_result = []
            vgg19_result = []


            cnt = 1

            for im in sorted(os.listdir(out_path)):
                pdf_name.append(pdf_file)
                #print(cnt)
                print(cnt,'  ',im)
                cnt = cnt + 1
                img = image.load_img(os.path.join(out_path,im),
                                 target_size=(224,224))
                #plt.imshow(img)
                #plt.show()

                x = image.img_to_array(img)
                x = x/255
                x = np.expand_dims(x,axis=0)
                images = np.vstack([x])
                val = inception_model.predict(images)
                #print(val)

                a=np.argmax(val,axis=1)
                output_inception = dic[a[0]]
                #print(output_inception)
                file_name.append(im)

                #image_read = cv2.imread(os.path.join(in_path,im),0)
                pil_read = Image.open(os.path.join(out_path,im))
                pil_read_gray = ImageOps.grayscale(pil_read)
                img_rgb = np.array(pil_read_gray)


                _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

                nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
                sizes = stats[1:, -1] #get CC_STAT_AREA component
                img2 = np.zeros((labels.shape), np.uint8)

                for i in range(0, nlabels - 1):
                    if sizes[i] >= 50:   #filter small dotted regions
                        img2[labels == i + 1] = 255

                res = cv2.bitwise_not(img2)
            #     data = Image.fromarray(res)
            #     enh_con = ImageEnhance.Contrast(data)
            #     contrast = 3.0
            #     image_contrasted = enh_con.enhance(contrast)
                image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
                image_data_lower = " ".join(image_data.splitlines()).lower()

                stats = ImageStat.Stat(pil_read)
                #print(stats)

                y = []
                for band,name in enumerate(pil_read.getbands()):
                    x=sum(stats.extrema[band])
                    y.append(x)
                if sum(y)==765:
                    #print('good')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray)
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

                else:
                    #print('dark')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
                    #print(image_data_1_lower)



                if output_inception == 'Blank':
                    inception_result.append(output_inception)
                else:


                    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                        inception_result.append('New Document')
                    else:
                        inception_result.append('Not Blank')


                val_1 = vgg_model.predict(images)
                #print(val_1)

                a_1=np.argmax(val_1,axis=1)
                output_vgg = dic[a_1[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg == 'Blank':
                    vgg_result.append(output_vgg)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg > 80 or fuzz_ratio_vgg_1 > 80:
                        vgg_result.append('New Document')
                    else:
                        vgg_result.append('Not Blank')


                val_2 = vgg19_model.predict(images)
                #print(val_1)

                a_2=np.argmax(val_2,axis=1)
                output_vgg19 = dic[a_2[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg19 == 'Blank':
                    vgg19_result.append(output_vgg19)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg19 = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg19_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg19 > 80 or fuzz_ratio_vgg19_1 > 80:
                        vgg19_result.append('New Document')
                    else:
                        vgg19_result.append('Not Blank')



                fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                    ocr_result.append('New Document')

                elif len(image_data_1_lower) < 15:
                    ocr_result.append('Blank')

                else:
                    ocr_result.append('Not Blank')


            df = pd.DataFrame({'File_Name':pdf_name,'Pages':file_name,'Inception_result':inception_result,
                               'VGG_result':vgg_result,'VGG19_result':vgg19_result,
                               'OCR_Result':ocr_result})
            #display(df)
            df['Confidence']=(df['Inception_result']==df['VGG_result'])&(df['Inception_result']==df['VGG19_result'])&(df['Inception_result']==df['OCR_Result'])

            df['status'] = df['Inception_result']
            df['Confidence'] = df['Confidence'].replace(True,'Yes')
            df['Confidence'] = df['Confidence'].replace(False,'No')

            pd.set_option('display.max_rows',1000)
            display(df)

            final_df = df[['File_Name','Pages','status','Confidence']]
            display(final_df)

            final_df.to_excel(os.path.join(excel_path,os.path.splitext(zip_file)[0],
                                           str(os.path.splitext(pdf_file)[0])+'.xlsx'))            

end = time.time()

print(end - start)

    



2022-02-17 10:44:53.623862
1507-1513.pdf
No of Images:  4
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg


,File_Name,Pages,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,1507-1513.pdf,1001.jpeg,New Document,New Document,New Document,New Document,Yes,New Document
1,1507-1513.pdf,1002.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
2,1507-1513.pdf,1003.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,1507-1513.pdf,1004.jpeg,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,File_Name,Pages,status,Confidence
0,1507-1513.pdf,1001.jpeg,New Document,Yes
1,1507-1513.pdf,1002.jpeg,Not Blank,Yes
2,1507-1513.pdf,1003.jpeg,Not Blank,Yes
3,1507-1513.pdf,1004.jpeg,Not Blank,Yes


38.62652015686035


# Zip with selecting pdf files folder


In [4]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
import zipfile
from pathlib import Path

from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt


start = time.time()
#print(start)
print(datetime.datetime.now())

#pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'
zip_file = 'test2 (copy).zip'

zip_file_path = '/home/vimalkumar/Documents/stewart/zip'
zip_path = '/home/vimalkumar/Documents/stewart/Pdf files'

os.makedirs(os.path.join(zip_path,os.path.splitext(zip_file)[0]))

with zipfile.ZipFile(os.path.join(zip_file_path,zip_file),"r") as zip_ref:
    #print(zip_ref)
    zip_ref.extractall(os.path.join(zip_path,os.path.splitext(zip_file)[0]))
    names = zip_ref.namelist()
    print(names)
    zip_folder = names[0][:-1]
    print(zip_folder)
    
file_path = '/home/vimalkumar/Documents/stewart/Pdf files'
images_path = '/home/vimalkumar/Documents/stewart/Images'
#excel_path = '/home/vimalkumar/Documents/stewart/excel_output'
excel_path = '/home/vimalkumar/Documents/stewart/zip'

os.makedirs(os.path.join(images_path,os.path.splitext(zip_file)[0]))
#os.makedirs(os.path.join(excel_path,os.path.splitext(zip_file)[0]))

pdf_name = []
file_name = []
inception_result = []
vgg_result = []
ocr_result = []
vgg19_result = []

for pdf_file in sorted(os.listdir(os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder))):
    if pdf_file.endswith('.pdf')or pdf_file.endswith('.PDF')or pdf_file.endswith('.Pdf'):
        print(pdf_file)
        in_path = os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder,pdf_file)
        out_path = os.path.join(images_path,os.path.splitext(zip_file)[0],
                                str(os.path.splitext(pdf_file)[0]))

        os.makedirs(out_path)
        
        file_size=Path(os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder,pdf_file)).stat().st_size
        
        if file_size != 0:
            doc = fitz.open(in_path)

            print('No of Images: ',len(doc))



            for i in range(len(doc)):
                doc = fitz.open(in_path)
                page = doc.load_page(i)  # number of page
                pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
                f_name = str(i+1000 + 1)       
                pix.pil_save(os.path.join(out_path,f_name+'.jpeg'))

            dic = {0:'Blank',1:'Not Blank'}

            inception_model = keras.models.load_model('stewart_inception_model.h5')
            vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')
            vgg19_model = keras.models.load_model('stewart_vgg19_model_1.h5')


#             pdf_name = []
#             file_name = []
#             inception_result = []
#             vgg_result = []
#             ocr_result = []
#             vgg19_result = []


            cnt = 1

            for im in sorted(os.listdir(out_path)):
                pdf_name.append(pdf_file)
                file_name.append(cnt)
                #print(cnt)
                print(cnt,'  ',im)
                cnt = cnt + 1
                img = image.load_img(os.path.join(out_path,im),
                                 target_size=(224,224))
                #plt.imshow(img)
                #plt.show()

                x = image.img_to_array(img)
                x = x/255
                x = np.expand_dims(x,axis=0)
                images = np.vstack([x])
                val = inception_model.predict(images)
                #print(val)

                a=np.argmax(val,axis=1)
                output_inception = dic[a[0]]
                #print(output_inception)
                #file_name.append(im)

                #image_read = cv2.imread(os.path.join(in_path,im),0)
                pil_read = Image.open(os.path.join(out_path,im))
                pil_read_gray = ImageOps.grayscale(pil_read)
                img_rgb = np.array(pil_read_gray)


                _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

                nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
                sizes = stats[1:, -1] #get CC_STAT_AREA component
                img2 = np.zeros((labels.shape), np.uint8)

                for i in range(0, nlabels - 1):
                    if sizes[i] >= 50:   #filter small dotted regions
                        img2[labels == i + 1] = 255

                res = cv2.bitwise_not(img2)
            #     data = Image.fromarray(res)
            #     enh_con = ImageEnhance.Contrast(data)
            #     contrast = 3.0
            #     image_contrasted = enh_con.enhance(contrast)
                image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
                image_data_lower = " ".join(image_data.splitlines()).lower()

                stats = ImageStat.Stat(pil_read)
                #print(stats)

                y = []
                for band,name in enumerate(pil_read.getbands()):
                    x=sum(stats.extrema[band])
                    y.append(x)
                if sum(y)==765:
                    #print('good')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray)
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

                else:
                    #print('dark')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
                    #print(image_data_1_lower)



                if output_inception == 'Blank':
                    inception_result.append(output_inception)
                else:


                    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                        inception_result.append('New Document')
                    else:
                        inception_result.append('Not Blank')


                val_1 = vgg_model.predict(images)
                #print(val_1)

                a_1=np.argmax(val_1,axis=1)
                output_vgg = dic[a_1[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg == 'Blank':
                    vgg_result.append(output_vgg)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg > 80 or fuzz_ratio_vgg_1 > 80:
                        vgg_result.append('New Document')
                    else:
                        vgg_result.append('Not Blank')


                val_2 = vgg19_model.predict(images)
                #print(val_1)

                a_2=np.argmax(val_2,axis=1)
                output_vgg19 = dic[a_2[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg19 == 'Blank':
                    vgg19_result.append(output_vgg19)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg19 = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg19_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg19 > 80 or fuzz_ratio_vgg19_1 > 80:
                        vgg19_result.append('New Document')
                    else:
                        vgg19_result.append('Not Blank')



                fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                    ocr_result.append('New Document')

                elif len(image_data_1_lower) < 15:
                    ocr_result.append('Blank')

                else:
                    ocr_result.append('Not Blank')


df = pd.DataFrame({'File_Name':pdf_name,'Page_Number':file_name,'Inception_result':inception_result,
                               'VGG_result':vgg_result,'VGG19_result':vgg19_result,
                               'OCR_Result':ocr_result})
#display(df)
df['Confidence']=(df['Inception_result']==df['VGG_result'])&(df['Inception_result']==df['VGG19_result'])&(df['Inception_result']==df['OCR_Result'])

df['status'] = df['Inception_result']
df['Confidence'] = df['Confidence'].replace(True,'Yes')
df['Confidence'] = df['Confidence'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

final_df = df[['File_Name','Page_Number','status','Confidence']]
display(final_df)

final_df.to_excel(os.path.join(excel_path,
                               str(os.path.splitext(zip_file)[0])+'.xlsx'),index=False)            

end = time.time()

print(end - start)

    



2022-02-17 13:11:45.407863
['part1/', 'part1/1507-1513.pdf', 'part1/Batch 17 (1).pdf']
part1
1507-1513.pdf
No of Images:  4
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
Batch 17 (1).pdf
No of Images:  24
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg


,File_Name,Page_Number,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,1507-1513.pdf,1,New Document,New Document,New Document,New Document,Yes,New Document
1,1507-1513.pdf,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
2,1507-1513.pdf,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,1507-1513.pdf,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 17 (1).pdf,1,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 17 (1).pdf,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
6,Batch 17 (1).pdf,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
7,Batch 17 (1).pdf,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
8,Batch 17 (1).pdf,5,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 17 (1).pdf,6,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,File_Name,Page_Number,status,Confidence
0,1507-1513.pdf,1,New Document,Yes
1,1507-1513.pdf,2,Not Blank,Yes
2,1507-1513.pdf,3,Not Blank,Yes
3,1507-1513.pdf,4,Not Blank,Yes
4,Batch 17 (1).pdf,1,Not Blank,Yes
5,Batch 17 (1).pdf,2,Not Blank,Yes
6,Batch 17 (1).pdf,3,Not Blank,Yes
7,Batch 17 (1).pdf,4,Not Blank,Yes
8,Batch 17 (1).pdf,5,Not Blank,Yes
9,Batch 17 (1).pdf,6,Not Blank,Yes


82.77011299133301


In [18]:
with zipfile.ZipFile('/home/vimalkumar/Documents/stewart/zip/test1.zip', 'r') as f:
    names = f.namelist()

In [21]:
names[0][:-1]

'part1'